In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 개요
[참고한 사이트](https://www.kaggle.com/code/heyrobin/spaceship-titanic-a-cosmic-mystery) SpaceTitanic Kaggle Competition 링크 글을 참고하여 코드를 따라쓰며 필자가 해석한 부분을 설명하면서 머신러닝에 대해 이해하기 위한 목적으로 작성하게 되었습니다.

<strong>Data</strong> - 승객의 약 3분의 2(~8700)에 대한 개인 기록은 훈련 데이터로 사용됩니다.
<ul><li><code>PassengerId</code> - 각 승객의 고유 ID. 각 ID는 <code>gggg_pp</code> 형식을 취하며, 여기서 <code>gggg</code>는 승객이 함께 여행하는 그룹을 나타내며, <code>pp</code>는 그룹 내 번호를 나타냅니다. 그룹의 사람들은 대게 가족 구성원이지만 항상 그렇만은 않습니다.</li>
<li><code>HomePlanet</code> - 승객이 출발한 행성으로, 일반적으로 그들의 영구 거주지 행성입니다.</li>
<li><code>CryoSleep</code> - 객이 항해가 진행되는 동안 가사상태로 전환되도록 선택했는지 여부를 나타냅니다. 저온 수면 중인 승객들은 객실에 있게 된다.</li>
<li><code>Cabin</code> - 승객이 머물고 있는 객실 번호입니다. 갑판/번호/측면<code>deck/num/side</code>형식을 취합니다. 여기서 측면<code>side</code>은 좌측이 <code>P</code>, <em>Port</em> 또는 우측이 <code>S</code>, <em>Starboard</em>가 될 수 있습니다.</li>
<li><code>Destination</code> - 승객이 정박할 목적지 행성입니다.</li>
<li><code>Age</code> - 승객의 나이를 나타냅니다.</li>
<li><code>VIP</code> - 승객이 항해 중 특별 VIP 서비스를 위해 비용을 지불했는지 여부를 나타낸 것 입니다.</li>
<li><code>RoomService</code>, <code>FoodCourt</code>, <code>ShoppingMall</code>, <code>Spa</code>, <code>VRDeck</code> - 승객들이 <em>Spaceship Titanic</em>의 많은 고급 편의시설에서 청구한 금액입니다. </li>
<li><code>Name</code> - 승객들의 이름을 나타냅니다.</li>
<li><code>Transported</code> - 승객이 다른 차원으로 이송되었는지 여부. 예측하려는 대상 열입니다.</li></ul></li>

# Step 1. 라이브러리 및 데이터 불러오기
- 본 프로젝트 수행을 위한 필수 라이브러리를 불러온다.

In [ ]:
'''Analysis(분석)'''

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt

'''Pre-Processing(데이터 전처리)'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

'''Model-Metrics(모형 개발 및 평가)'''
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore') # 경고 메세지 숨기기 참고사이트 [https://rfriend.tistory.com/346]

'''Dataset'''
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

## Model-Metrics 라이브러리 참고 사이트
- [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
- [XGBClassifier](https://xgboost.readthedocs.io/en/stable/python/python_api.html)
- [LGBMClassifier](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html)
- [CatBoostClassifier](https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier)
- [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
- [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
- [Classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html), [Accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html), [Confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

- 라이브러리 버전 확인

In [ ]:
print(np.__version__)
print(pd.__version__)
print(sns.__version__)

- 데이터 확인
    + 훈련데이터는 8,693개, 테스트 데이터는 4,277개로 구성이 되어 있음
    + 독립변수 컬럼 갯수는 13개이며, 종속 변수는 1개로 구성되어 있음

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
train.info()

- 컬럼별 레이블의 개수 확인
    + data_Frame.nunique() = 데이터셋에 존재하는 컬럼별 레이블의 개수를 확인하기 위해 사용

In [ ]:
train.nunique()

- 대회주제
    + Transported Column 설명을 보면, 다른 차원으로 이동하기 위한, 각 승객의 탑승을 했는지 못했는지를 예측하는 문제이며, 평가지표는 정확도로 판정함.
- 분류 정확도(Classification Accuracy)의 설명은 [Classification Accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy)에서 확인할 수 있다. 

# Step 2. 탐색적 자료 분석 (EDA)
- 데이터 시각화
- 산점도, 막대 그래프 등
- 그래프 해석 및 설명
- 약간의 데이터 전처리

- HomePlanet

In [ ]:
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
init_notebook_mode()
import plotly.graph_objs as go

cnt_srs = train['HomePlanet'].value_counts()

trace = go.Scatter(
    x = cnt_srs.index,
    y = cnt_srs.values,
    mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        sizeref = 20,
        size = cnt_srs.values,
        color = ['#1D7595','#B9B596','#864D29']

    ),
)

layout = go.Layout(
    title = '<b>Home Planets</b>', 
    title_x = 0.5,
    titlefont = dict(size =20, color='black', family='Space Mono'),
    plot_bgcolor = 'rgba(0,0,0,0)'
)

data = [trace]
fig = go.Figure(data = data, layout = layout)
fig.update_xaxes(showline = True, linewidth = 2,
                 #linecolor = colors[3]
                )
fig.update_yaxes(showline = True, linewidth = 2,
                 #linecolor = colors[3]
                )
py.iplot(fig, filename = "Size of the Company/Clan")
fig.show()

    + Earth: 4602
    + Europa: 2131
    + Mars: 1759

- Destination

In [ ]:
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
init_notebook_mode()
import plotly.graph_objs as go

cnt_srs = train['Destination'].value_counts()

trace = go.Scatter(
    x = cnt_srs.index,
    y = cnt_srs.values,
    mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        sizeref = 20,
        size = cnt_srs.values,
        color = ['#048B95','#A1231F','#602F58']
    ),
)



layout = go.Layout(
    title = '<b>Destination</b>', 
    title_x = 0.5,
    titlefont = dict(size = 20, color = 'black', family = 'Space Mono'),
    
    plot_bgcolor = 'rgba(0,0,0,0)'
)

fig = go.Figure(data = [trace], layout = layout)

sources = "https://assets.stickpng.com/images/580b585b2edbce24c47b2d2a.png"
# add images

fig.add_layout_image(
        source=sources,
        xref = "x domain",
        yref = "y domain",
        x = 0.2,
        y = 0.8,
        xanchor = "right",
        yanchor = "top",
        sizex = 1,
        sizey = 0.7,
        opacity = False
    )

fig.update_xaxes(showline=True, linewidth=2)
fig.update_yaxes(showline=True, linewidth=2)
fig.show()

    + TRAPPIST-1e: 5915
    + 55 Cancri e: 1800
    + PSO J318.5-22: 796

- Age Distribution

In [ ]:
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
init_notebook_mode()
import plotly.graph_objs as go

trace = go.Histogram(x = train['Age'],
                            histnorm = 'percent',
                            xbins = dict(
                                start = 0,
                                end = 100,
                                size = 1),
                            marker_color = '#048B95',
                            )


layout = go.Layout(
    title = '<b>Age Distribution</b>',
    title_x = 0.5,
    titlefont = dict(size = 20, color = 'black', family = 'Space Mono'),
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    xaxis_title = 'Age',
    yaxis_title = 'Percent'
)


fig = go.Figure(data = [trace], layout = layout)

fig.update_xaxes(showline = True, linewidth = 2)
fig.update_yaxes(showline = True, linewidth = 2)

py.iplot(fig, filename = "planets")
fig.show()

# Step 3. 데이터 전처리
- Feature Engineering
- ML 모형을 돌리기 위해 표준화 등
- 파생변수 (도출 변수)
    + 왜 이 변수를 만들었는지에 대한 설명 필요

- 필요없는 특성 정리
    + drop(inplce = True) : 기존 프레임에 변경된 설정으로 덮어쓰겠다는 의미, 그렇기 때문에 다시 데이터 프레임을 가져왔을때 출력되지 않음.

In [ ]:
# taking target variable and droping it
y = train['Transported']
train = train.drop('Transported', axis = 1)
train

- 불필요한 변수 제거
    + Name, PassengerId, Cabin
    + Cabin은 모형 개선 시, 문자열 처리를 통해 추가 변수를 도출 할수는 있지만, 일단 제거한다.

In [ ]:
#cols_to_drop = ['Name', 'PassengerId']
cols_to_drop = ['Name', 'PassengerId', 'Cabin']
def drop(train,test,cols):
    train.drop(cols, axis = 1, inplace = True)
    test.drop(cols, axis = 1, inplace = True)
    print('sucessfully droped features')
    
drop(train, test, cols_to_drop)

In [ ]:
train

In [ ]:
#train[['Deck', 'Num', 'Side']] = train['Cabin'].str.split('/', expand = True)
#test[['Deck', 'Num', 'Side']] = train['Cabin'].str.split('/', expand=True)

- 결측치에 넣을 categorical and numerical feature 파생 변수 생성

In [ ]:
# getting categorical and numerical feature
cat_col = [col for col in train.columns if train[col].dtype not in ["float64"]]
num_col = [col for col in train.columns if train[col].dtype in ["float64"]]

## **Handling Missing Values /Imputation (결측치)**
- 결측치 데이터를 추가하도록 한다. 결측치를 추가하기 위해 필자는 [Sklearn.IterativeImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html) 클래스를 사용하였다.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(random_state = 0, max_iter = 10, initial_strategy = 'mean')

train[num_col] = imputer.fit_transform(train[num_col])
test[num_col] = imputer.fit_transform(test[num_col])

In [ ]:
train[cat_col] = train[cat_col].fillna('Nan')
test[cat_col] = test[cat_col].fillna('Nan')

In [ ]:
train[cat_col]

In [ ]:
test[num_col]

## Categorical Feature Encoding
- 머신러닝 알고리즘은 수식으로 구성이 되어 있기 때문에 문자열의 경우 인코딩으로 변환을 주어야 한다. 

- 필자의 경우 [Labelencoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) 모델을 사용하여 인코딩 진행
    + 문자열을 데이터 프레임형식으로 인코딩 후 훈련 진행

In [ ]:
def label_encoder(df, df2, columns):
    for col in columns:
        df[col] = df[col].astype(str)
        df2[col] = df2[col].astype(str)
        df[col] = LabelEncoder().fit_transform(df[col])
        df2[col] = LabelEncoder().fit_transform(df2[col])
    return df

label_encoder(train,test,cat_col)
type(label_encoder(train,test,cat_col))

# Step 4. 머신러닝 모형 개발
- 모형에 대한 설명 필요
- 모형을 1-2개 사용
- 교차 검증
- 하이퍼 파라미터 튜닝

- 훈련데이터와 검증데이터 분리 및 교차 검증 3회 적용.

In [ ]:
X = train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

- 적용 할 모형(모델)별 스코어 확인
    + Dictionary 형식으로 name 과 values 값을 형성하여 for 문 사용
    + 선형회귀, K-최근접이웃, 서포트벡터, 결정트리, XGB, LightGBM 클래스를 사용하여 모형(모델)별 정확도 비교

In [ ]:
models = {"LogisticClassifier":LogisticRegression(solver='liblinear'),
          "KNearest": KNeighborsClassifier(),
          "Support Vector Classifier": SVC(),
          "DecisionTreeClassifier": DecisionTreeClassifier(),
          "XGBClassifier":XGBClassifier(),
          "LightLGBM Classifier":LGBMClassifier(),
          #"CatBoostClassifier":CatBoostClassifier()
          }

In [ ]:
for name, vals in models.items():
    model = vals.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test,y_pred)
    print(f'{name} accuracy is {score}')

- LightLGBM Classifier 모델이 0.789로 가장 높은 스코어를 가진것을 확인

# Step 5. 모형 평가
- 훈련데이터 쪼갠다. 훈련데이터 + 검증데이터 분리
- 정확도 비교
- 혼동 행렬 (Confusion Matrix) 설명 ** [오분류비용](http://contents.kocw.net/KOCW/document/2014/Chungbuk/choisanghyun/6.pdf)

- optuna 사용하여 최적의 하이퍼파라미터 확인
    + 그리드 서치는 시간이 조금 오래 걸릴 뿐만 아니라, 하이퍼 파라미터 값을 직접 지정해주어야 한다는 단점이 있다. Optuna도 물론 값을 지정해주어야 하지만, 조금 더 러프하게 값을 지정해주면, Optuna가 범위내에서 자동탐색을 통해 최적의 하이퍼파라미터를 도출해준다는 점과 시간이 조금 더 빠르다는 점에서 GridSearchCV보다 좋은 것 같다. [설명 출처 사이트](https://hojjimin-statistic.tistory.com/27)

In [ ]:
'''import optuna
from sklearn.model_selection import cross_val_score

# LGBM 하이퍼 파라미터 값 지정
def objective(trial):
             
      n_estimators = trial.suggest_int("n_estimators", 50, 200)
      learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
      num_leaves = trial.suggest_int("num_leaves", 20, 3000, step=20)
      max_depth = trial.suggest_int("max_depth", 3, 12)
      min_data_in_leaf = trial.suggest_int("min_data_in_leaf", 200, 10000, step=100)
      lambda_l1 = trial.suggest_int("lambda_l1", 0, 100, step=5)
      lambda_l2 = trial.suggest_int("lambda_l2", 0, 100, step=5)
      min_gain_to_split = trial.suggest_float("min_gain_to_split", 0, 15)
      bagging_fraction = trial.suggest_float("bagging_fraction", 0.2, 0.95, step=0.1)
      bagging_freq = trial.suggest_categorical("bagging_freq", [1])
      feature_fraction = trial.suggest_float("feature_fraction", 0.2, 0.95, step=0.1)
      
      # LGBM 클래스로 학습 모델 설정
      lgbc = LGBMClassifier(n_estimators=n_estimators,
                            #max_depth=max_depth,
                            learning_rate=learning_rate,
                            #num_leaves =num_leaves,
                            max_depth=max_depth,
                            min_data_in_leaf=min_data_in_leaf,
                            #lambda_l1=lambda_l1,
                            #lambda_l2=lambda_l2,
                            #min_gain_to_split=min_gain_to_split,
                            #bagging_fraction=bagging_fraction,
                            #bagging_freq=bagging_freq,
                            #feature_fraction=feature_fraction
                           )
    
      return cross_val_score(lgbc,
                             X,
                             y,
                             n_jobs=-1,
                             cv=3).mean()                             
study = optuna.create_study(direction='maximize') # MAE(평균절대오차)가 최대가 되는 방향으로 학습을 진행
study.optimize(objective, n_trials=100)           # n_trials 지정해주지 않으면, 무한 반복

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))

print("Best hyperparameters: {}".format(trial.params))

best_params = trial.params

'''

- Accuracy: 0.7977694653941446
- Best hyperparameters: {'n_estimators': 68, 'learning_rate': 0.14320356701453069, 'num_leaves': 2220, 'max_depth': 7, 'min_data_in_leaf': 200, 'lambda_l1': 90, 'lambda_l2': 45, 'min_gain_to_split': 1.516682995396741, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.5}

In [ ]:
#optuna.visualization.plot_optimization_history(study)
#optuna.visualization.plot_slice(study)

In [ ]:
lgbc = LGBMClassifier(
    #**best_params
).fit(X_train, y_train)
y_pred = lgbc.predict(X_test)
score=accuracy_score(y_test,y_pred)
print(score) # 정확도 확인

- 혼동 행렬(confusion matrix)
    + 모델의 성능을 평가할때 사용되는 지표
    + 예측값이 실제 관측값을 얼마나 정확히 예측했는지 보여주는 행렬
    + [혼동 행렬 설명 참조](https://truman.tistory.com/179)
    + from sklearn.[metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) import confusion_matrix 모델 사용

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),
                annot = True,
                cbar= False,
                fmt='g',
               #cmap= theme
               )
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.title('Confusion-Matrix');

In [ ]:
TP = 964 # True Positive 분석 모델이 1로 예측했는데 실제 데이터도 1인 경우
FN = 325 # False Negative 분석 모델이 0로 예측했는데 실제 데이터는 1인 경우
FP = 224 # False Positive 분석 모델이 0로 예측했는데 실제 데이터는 0인 경우
TN = 1095 # True Negative 분석 모델이 1로 예측했는데 실제 데이터는 0인 경우
# Precision(정밀도): 모델이 1로 예측한 결과 중 실제 데이터도 1인 경우의 비율
P = TP / (TP + FP)
# Accurancy(정확도): 모델이 제대로 예측한 비율
A = (TP + TN) / (TP + FP + FN + TN)
# Error Rate(오차 비율): 1-Accyracy
ER = (FP + FN) / (TP + FP + FN + TN)
# False Poitive Rate(FPR, 거짓긍정률): 1-Specificity
FPR = FP / (FP + TN)
# Sensitivity(or Recall, 민감도): 실제 값이 1인 데이터 중 모델이 1로 예측한 비율
S = TP / (TP + FN)
# Specificity(특이도): 실제 값이 0인 데이터 중 모델이 0으로 예측한 비율
Spe = TN / (TN + FP)
# F1 Score (F-Measure): 정밀도와 재현율의 조화평균, 1에 가까울수록 모델의 예측 성능이 좋음 
F1 = 2 / ((1 / P) + (1 / S))
print('Precision:', P,
      'Accurancy(score):', A,
      'Error_Rate:', ER,
      'False_Positive_Rate:', FPR,
      'Sensitivity:', S,
      'Specificity:', Spe,
      'F1_Score:', F1, sep='\n')

- [위 참고 자료 출처_1](https://bibimnews.com/entry/%EB%A7%A4%ED%8A%B8%EB%A6%AD%EC%8A%A4%EB%A5%BC-%ED%86%B5%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%AA%A8%EB%8D%B8-%ED%8F%89%EA%B0%80-%EA%B8%B0%EB%B2%95-%ED%98%BC%EB%8F%99%ED%96%89%EB%A0%ACConfusion-Matrix)
- [위 참고 자료 출처_2](https://yogyui.tistory.com/entry/Confusion-Matrix-%ED%98%BC%EB%8F%99%ED%96%89%EB%A0%AC)

# 참고
- 다른 사람의 code 설명을 쭉 따라치고 필자가 해석한 것을 추가로 작성함.
- Robin, Spaceship Titanic: A cosmic mystery , [링크 주소](https://www.kaggle.com/code/heyrobin/spaceship-titanic-a-cosmic-mystery)

# Step 6. 제출

In [ ]:
sub = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
sub['Transported'] = lgbc.predict(test)
sub.to_csv('submission.csv',index=False)
sub.head()